In [1]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import sqlite3

In [2]:
stopandsearchdf = pd.read_parquet('stopandsearch.paraquet')

In [3]:
stopandsearchdf

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
178930,Person search,2016-05-31T23:01:00+00:00,0.0,NaN,51.455989,-0.043536,Male,10-17,Black or Black British - African (B2),Black,Controlled drugs,Nothing found - no further action,NaN,NaN
178931,Person and Vehicle search,2016-05-31T23:05:00+00:00,0.0,NaN,51.475561,-0.075330,Male,None,Not Stated (NS),Black,Controlled drugs,Nothing found - no further action,NaN,NaN
178932,Person search,2016-05-31T23:07:00+00:00,0.0,NaN,51.533030,-0.203277,Male,18-24,Black or Black British - Caribbean (B1),Black,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN
178933,Person search,2016-05-31T23:10:00+00:00,0.0,NaN,51.553184,-0.176043,Male,18-24,Not Stated (NS),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN
178934,Person search,2016-05-31T23:10:00+00:00,0.0,NaN,51.553184,-0.176043,Female,18-24,White - Any other White ethnic background (W9),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533191,Person search,2023-03-31T22:43:00+00:00,0.0,NaN,51.578389,0.180957,Male,over 34,Other ethnic group - Any other ethnic group,Asian,Offensive weapons,A no further action disposal,NaN,NaN
1533192,Person search,2023-03-31T22:44:00+00:00,0.0,NaN,51.516295,-0.129487,Male,25-34,Other ethnic group - Not stated,Black,Controlled drugs,A no further action disposal,NaN,NaN
1533193,Person search,2023-03-31T22:44:00+00:00,0.0,NaN,51.545724,-0.011334,Male,25-34,Black/African/Caribbean/Black British - African,Black,Controlled drugs,Community resolution,NaN,NaN
1533194,Person search,2023-03-31T22:48:00+00:00,0.0,NaN,51.528633,-0.125729,Female,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,A no further action disposal,NaN,NaN


In [4]:
stopandsearchdf['Outcome'].unique()

stopandsearchdf['Further_outcome'] = stopandsearchdf['Outcome'].apply(lambda x: 1 if x != 'Nothing found - no further action' else 0)

stopandsearchdf

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Further_outcome
178930,Person search,2016-05-31T23:01:00+00:00,0.0,NaN,51.455989,-0.043536,Male,10-17,Black or Black British - African (B2),Black,Controlled drugs,Nothing found - no further action,NaN,NaN,0
178931,Person and Vehicle search,2016-05-31T23:05:00+00:00,0.0,NaN,51.475561,-0.075330,Male,None,Not Stated (NS),Black,Controlled drugs,Nothing found - no further action,NaN,NaN,0
178932,Person search,2016-05-31T23:07:00+00:00,0.0,NaN,51.533030,-0.203277,Male,18-24,Black or Black British - Caribbean (B1),Black,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0
178933,Person search,2016-05-31T23:10:00+00:00,0.0,NaN,51.553184,-0.176043,Male,18-24,Not Stated (NS),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0
178934,Person search,2016-05-31T23:10:00+00:00,0.0,NaN,51.553184,-0.176043,Female,18-24,White - Any other White ethnic background (W9),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533191,Person search,2023-03-31T22:43:00+00:00,0.0,NaN,51.578389,0.180957,Male,over 34,Other ethnic group - Any other ethnic group,Asian,Offensive weapons,A no further action disposal,NaN,NaN,1
1533192,Person search,2023-03-31T22:44:00+00:00,0.0,NaN,51.516295,-0.129487,Male,25-34,Other ethnic group - Not stated,Black,Controlled drugs,A no further action disposal,NaN,NaN,1
1533193,Person search,2023-03-31T22:44:00+00:00,0.0,NaN,51.545724,-0.011334,Male,25-34,Black/African/Caribbean/Black British - African,Black,Controlled drugs,Community resolution,NaN,NaN,1
1533194,Person search,2023-03-31T22:48:00+00:00,0.0,NaN,51.528633,-0.125729,Female,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,A no further action disposal,NaN,NaN,1


In [5]:
# Convert 'Date' column to datetime format
stopandsearchdf['Date'] = pd.to_datetime(stopandsearchdf['Date'])

# Extract year and month into new columns
stopandsearchdf['Year'] = stopandsearchdf['Date'].dt.year
stopandsearchdf['Month'] = stopandsearchdf['Date'].dt.month

# Remove the 'Date' column
stopandsearchdf = stopandsearchdf.drop('Date', axis=1)
#outcome =  str(input("Enter the stop and search outcome to filter by: "))
#stopandsearchdf = stopandsearchdf[stopandsearchdf['Outcome'] != outcome].copy()


In [6]:
stopandsearchdf['season'] =  pd.cut(stopandsearchdf['Month'], bins=[0, 4, 8, 12], labels=[1, 2, 3])
stopandsearchdf

,Type,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Further_outcome,Year,Month,season
178930,Person search,0.0,NaN,51.455989,-0.043536,Male,10-17,Black or Black British - African (B2),Black,Controlled drugs,Nothing found - no further action,NaN,NaN,0,2016,5,2
178931,Person and Vehicle search,0.0,NaN,51.475561,-0.075330,Male,None,Not Stated (NS),Black,Controlled drugs,Nothing found - no further action,NaN,NaN,0,2016,5,2
178932,Person search,0.0,NaN,51.533030,-0.203277,Male,18-24,Black or Black British - Caribbean (B1),Black,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0,2016,5,2
178933,Person search,0.0,NaN,51.553184,-0.176043,Male,18-24,Not Stated (NS),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0,2016,5,2
178934,Person search,0.0,NaN,51.553184,-0.176043,Female,18-24,White - Any other White ethnic background (W9),White,Articles for use in criminal damage,Nothing found - no further action,NaN,NaN,0,2016,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533191,Person search,0.0,NaN,51.578389,0.180957,Male,over 34,Other ethnic group - Any other ethnic group,Asian,Offensive weapons,A no further action disposal,NaN,NaN,1,2023,3,1
1533192,Person search,0.0,NaN,51.516295,-0.129487,Male,25-34,Other ethnic group - Not stated,Black,Controlled drugs,A no further action disposal,NaN,NaN,1,2023,3,1
1533193,Person search,0.0,NaN,51.545724,-0.011334,Male,25-34,Black/African/Caribbean/Black British - African,Black,Controlled drugs,Community resolution,NaN,NaN,1,2023,3,1
1533194,Person search,0.0,NaN,51.528633,-0.125729,Female,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,A no further action disposal,NaN,NaN,1,2023,3,1


In [7]:
# Read the GeoJSON file
lsoa_data = gpd.read_file("london.geojson")

# Convert the coordinates in the stop and search data to Point objects
stopandsearchdf['geometry'] = [Point(xy) for xy in zip(stopandsearchdf['Longitude'], stopandsearchdf['Latitude'])]
stopandsearchdf = gpd.GeoDataFrame(stopandsearchdf, geometry='geometry')

# Perform the spatial join
stopandsearchdf_with_lsoa = gpd.sjoin(stopandsearchdf, lsoa_data, how='left', op='within')

# Extract the LSOA code from the joined data
stopandsearchdf_with_lsoa['LSOA_code'] = stopandsearchdf_with_lsoa['lsoa11cd']

# Save the result or use it for further analysis
#stopandsearchdf_with_lsoa.to_csv("stopandsearch_with_lsoa.csv")


/Users/drugarobert/opt/anaconda3/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/Users/drugarobert/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/1_/1c03qm8s7hg966bpwbd0nfs80000gn/T/ipykernel_3703/1954923633.py:9: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  stopandsearchdf_with_lsoa = gpd.sjoin(stopandsearchdf, lsoa_data, how='left', op='within')


In [ ]:
stopandsearchdf_with_lsoa

In [ ]:
london_lsoa = gpd.read_file('London.geojson')

# Filter the LSOA data for Barnet
barnet_lsoa = london_lsoa[london_lsoa['lsoa11nm'].str.contains('Barnet')]


In [ ]:
barnet_lsoa

In [ ]:
# We focus on Hendon and West Hendon wards
areas = ['Barnet']
lsoa_codes = ['031A', '031B', '031C', '032A', '032B', '032C', '032D', '032E', '034A', '034B', '034D', '031D', '032F', '034C', '039D', '036B', '036C', '036D', '036F']
combined_list = [area + ' ' + code for area in areas for code in lsoa_codes]
# Extract the data for stop and search
hendon_stop_search = barnet_lsoa[barnet_lsoa['lsoa11nm'].isin(combined_list)]


In [ ]:
hendon_stop_search

In [ ]:
stopandsearchdf_with_lsoa_barnet_hendon = stopandsearchdf_with_lsoa[stopandsearchdf_with_lsoa['lsoa11cd'].isin(hendon_stop_search['lsoa11cd'].values)]

stopandsearchdf_with_lsoa_barnet = stopandsearchdf_with_lsoa[stopandsearchdf_with_lsoa['lsoa11cd'].isin(barnet_lsoa['lsoa11cd'].values)]

stopandsearchdf_with_lsoa_barnet_hendon.to_csv("stopandsearch_with_lsoa_hendon.csv")



In [ ]:
perc_hendon = sum(stopandsearchdf_with_lsoa_barnet_hendon['Further_outcome'])/len(stopandsearchdf_with_lsoa_barnet_hendon['Further_outcome'])

In [ ]:
print("The percentage of stop and searches that lead to a further outcome in Hendon is ",perc_hendon * 100,'%')

In [ ]:
perc_barnet = sum(stopandsearchdf_with_lsoa_barnet['Further_outcome'])/len(stopandsearchdf_with_lsoa_barnet['Further_outcome'])




print("The percentage of stop and searches that lead to a further outcome in the whole Barnet is ",perc_barnet * 100,'%')

In [ ]:
stopandsearchdf['Outcome'].unique()